In [2]:
import numpy as np
import pandas as pd
import pybithumb
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from Funcs_CNN4 import obv, macd, cmo, rsi
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Scaler = MinMaxScaler()

dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)


def min_max_scaler(price):
    Scaler = MinMaxScaler()
    Scaler.fit(price)

    return Scaler.transform(price)


def low_high(Coin, input_data_length, ip_limit=None, trade_limit=None, crop_size=300, lowhigh_point=None):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    #   IP - Change
    if ip_limit is None:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')
    else:
        ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1', 'proxyon')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None, None, None

    ohlcv_excel['OBV'] = obv(ohlcv_excel)

    if lowhigh_point is not None:
        check_span = 20
        trade_state = [np.NaN] * len(ohlcv_excel)
        for i in range(check_span, len(ohlcv_excel) - check_span):
            #   저점
            if ohlcv_excel['close'][i - check_span:i].min() >= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                    if 1 not in trade_state[i - check_span:i]:
                        trade_state[i] = 1
                    else:
                        trade_state[i] = 0
                    # else:
                    #     trade_state[i] = 0
                else:
                    trade_state[i] = 0
            #   고점
            elif ohlcv_excel['close'][i - check_span:i].max() <= ohlcv_excel['close'][i]:
                if ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
                    # if ohlcv_excel['close'][i - check_span:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                    if 2 not in trade_state[i - check_span:i]:
                        trade_state[i] = 2
                    else:
                        trade_state[i] = 0
                # else:
                #     trade_state[i] = 0
                else:
                    trade_state[i] = 0

            #   거래 안함
            else:
                trade_state[i] = 0

        ohlcv_excel['trade_state'] = trade_state

        #   현재가가 이전 저점보다 0.1 이상 크지 않으면 거래하지 않는다.
        low_index = None
        for i in range(len(trade_state)):
            if trade_state[-i] == 1:
                low_index = i

    closeprice = ohlcv_excel['close'].iloc[-1]
    datetime_list = ohlcv_excel.index.values

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    scaled_close = min_max_scaler(ohlcv_data[:, [1]])
    prev_low = scaled_close[-low_index]
    pres_low = scaled_close[-1]
    if pres_low - prev_low < 0.1:
        return None, None, None, None

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        # price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # OBV = ohlcv_data[:, [-1]]
        #
        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_OBV = min_max_scaler(OBV)
        # print(scaled_MA60.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # print(x.shape)
        # print(ohlcv_data.shape)
        # quit()

        # if (x[-1][1] > 0.3) and (trade_limit is not None):
        #     return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(crop_size, len(ohlcv_data) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            scaled_price = min_max_scaler(group_x[:, :4])
            # scaled_volume = min_max_scaler(group_x[:, [4]])
            # scaled_OBV = min_max_scaler(group_x[:, [-1]])
            # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1) + sudden_death  # axis=1, 세로로 합친다
            x = scaled_price  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[-1:, 1:2])
            # print(group_x.shape)
            # print(group_x.max())
            dataX.append(group_x)  # dataX 리스트에 추가

        if len(dataX) < 100:
            return None, None, None, None

        # quit()
        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # closeprice = np.roll(np.array(list(map(lambda x: x[-1][[1]][0], X_test))), -1)
        # plt.plot(closeprice)
        # plt.show()

        return X_test, closeprice, min_max_scaler(ohlcv_data[crop_size:, [1]]), datetime_list


def made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=None, sudden_death=0):

    ohlcv_excel = pd.read_excel(dir + file, index_col=0)

    ohlcv_excel['MA20'] = ohlcv_excel['close'].rolling(20).mean()
    ohlcv_excel['CMO'] = cmo(ohlcv_excel)
    ohlcv_excel['OBV'] = obv(ohlcv_excel)
    ohlcv_excel['RSI'] = rsi(ohlcv_excel)
    macd(ohlcv_excel, short=15, long=60)

    # print(ohlcv_excel)
    # quit()

    #   이후 check_span 데이터와 현재 포인트를 비교해서 현재 포인트가 저가인지 고가인지 예측한다.
    #   진입, 저점, 고점, 거래 안함의 y_label 인 trade_state  >> [1, 2, 0]
    #   저점과 고점은 최대 3개의 중복 값을 허용한다.
    trade_state = [np.NaN] * len(ohlcv_excel)
    for i in range(len(ohlcv_excel) - check_span):
        #   저점
        if ohlcv_excel['close'][i + 1:i + 1 + check_span].min() >= ohlcv_excel['close'][i]:
            if ohlcv_excel['close'][i:i + 1 + check_span].value_counts().sort_index().iloc[0] <= 3:
                trade_state[i] = 1
            else:
                trade_state[i] = 0
        #   고점
        elif ohlcv_excel['close'][i + 1:i + 1 + check_span].max() <= ohlcv_excel['close'][i]:
            if ohlcv_excel['close'][i:i + 1 + check_span].value_counts().sort_index().iloc[-1] <= 3:
                trade_state[i] = 2
            else:
                trade_state[i] = 0
        #   거래 안함
        else:
            trade_state[i] = 0

    ohlcv_excel['trade_state'] = trade_state

    # ----------- dataX, dataY 추출하기 -----------#
    # print(ohlcv_excel.info())
    # quit()
    # ohlcv_excel.to_excel('test.xlsx')

    # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
    #   OBV : -CHECK_SPAN
    ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.MACD_Signal.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[1: -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.CMO.isna()): -check_span].astype(np.float)
    # ohlcv_data = ohlcv_excel.values[sum(ohlcv_excel.RSI.isna()): -check_span].astype(np.float)

    # print(pd.DataFrame(ohlcv_data).info())
    # print(pd.DataFrame(ohlcv_data).to_excel('test.xlsx'))
    # print(list(map(float, ohlcv_data[0])))
    # quit()

    #       출력되는 Y 데이터 길이를 동일화 시킨다.     #
    # ohlcv_data = ohlcv_data[-300:]

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        price = ohlcv_data[:, :4]
        volume = ohlcv_data[:, [4]]
        MA20 = ohlcv_data[:, [-8]]
        CMO = ohlcv_data[:, [-7]]
        OBV = ohlcv_data[:, [-6]]
        RSI = ohlcv_data[:, [-5]]
        MACD = ohlcv_data[:, [-4]]
        MACD_Signal = ohlcv_data[:, [-3]]
        MACD_OSC = ohlcv_data[:, [-2]]

        #   Flexible Y_data    #
        trade_state = ohlcv_data[:, [-1]]

        # scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_CMO = min_max_scaler(CMO)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_RSI = min_max_scaler(RSI)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        # x = scaled_price
        y = trade_state
        # print(x.shape, y_low.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇
        for i in range(crop_size, len(ohlcv_data)):  # 마지막 데이터까지 다 긇어모은다.
            group_x = ohlcv_data[i - crop_size: i]
            group_y = y[i]
            price = group_x[:, :4]
            volume = group_x[:, [4]]
            MA20 = group_x[:, [-8]]
            
            pricema = np.concatenate((price, MA20), axis=1)
            scaled_pricema = (pricema - pricema.min()) / (pricema.max() - pricema.min())
            
            CMO = group_x[:, [-7]]
            OBV = group_x[:, [-6]]
            RSI = group_x[:, [-5]]
            MACD = group_x[:, -4:-1]

            scaled_volume = min_max_scaler(volume)
            scaled_CMO = min_max_scaler(CMO)
            scaled_OBV = min_max_scaler(OBV)
            scaled_RSI = min_max_scaler(RSI)
            scaled_MACD = (MACD - MACD.min()) / (MACD.max() - MACD.min())
            x = np.concatenate((scaled_pricema[:, :4], scaled_volume, 
                                scaled_pricema[:, 4:], scaled_CMO, scaled_OBV, scaled_RSI,
                 scaled_MACD), axis=1)
            # x = scaled_x + sudden_death  # axis=1, 세로로 합친다
            group_x = x[-input_data_length:]
            # print(group_x[0])
            # quit()

            #   데이터 값에 결측치가 존재하는 경우 #
            if sum(sum(np.isnan(group_x))) > 0:
                continue

            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)

        if len(dataX) < 100:
            print('len(dataX) < 100')
            return None, None, None

        # X_test = np.array(dataX)
        # row = X_test.shape[1]
        # col = X_test.shape[2]
        #
        # X_test = X_test.astype('float32').reshape(-1, row, col, 1)
        # print(X_test.shape)

        #       Exstracting fiexd X_data       #
        sliced_ohlcv = ohlcv_data[crop_size:, :x.shape[1]]

        #                      Get Figure                     #
        if get_fig == 1:
            spanlist_low = []
            spanlist_high = []

            for m in range(len(trade_state)):
                if (trade_state[m] > 0.5) and (trade_state[m] < 1.5):
                    if m + 1 < len(trade_state):
                        spanlist_low.append((m, m + 1))
                    else:
                        spanlist_low.append((m - 1, m))

            for m in range(len(trade_state)):
                if (trade_state[m] > 1.5) and (trade_state[m] < 2.5):
                    if m + 1 < len(trade_state):
                        spanlist_high.append((m, m + 1))
                    else:
                        spanlist_high.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- 공통된 Chart 그리기 -----------#

            plt.subplot(211)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_low)):
                plt.axvspan(spanlist_low[i][0], spanlist_low[i][1], facecolor='m', alpha=0.5)

            plt.subplot(212)
            plt.plot(min_max_scaler(ohlcv_data[:, 1:2]), 'r', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')
            for i in range(len(spanlist_high)):
                plt.axvspan(spanlist_high[i][0], spanlist_high[i][1], facecolor='c', alpha=0.5)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
            # ----------- Chart 그리기 -----------#

        return dataX, dataY, sliced_ohlcv


if __name__ == '__main__':

    # ----------- Params -----------#
    input_data_length = 30
    model_num = 78

    #       Make folder      #
    try:
        os.mkdir('./Figure_data/%s_%s/' % (input_data_length, model_num))

    except Exception as e:
        pass
    check_span = 30
    get_fig = 0

    Made_X = []
    Made_Y = []

#     ohlcv_list = ['2020-01-10 ETH ohlcv.xlsx']

    for file in ohlcv_list:
        try:
            if int(file.split()[0].split('-')[1]) != 1:
                continue


            result = made_x(file, input_data_length, model_num, check_span, get_fig, crop_size=input_data_length)
            # result = low_high('dvp'.upper(), input_data_length)
            # print(result)
            # quit()

            # ------------ 데이터가 있으면 dataX, dataY 병합하기 ------------#
            if result is not None:
                if result[0] is not None:

                    Made_X += result[0]
                    Made_Y += result[1]

                # 누적 데이터량 표시
                print(file, len(Made_X))
                
        except Exception as e:
            pass

    # SAVING X, Y
    X = np.array(Made_X)
    Y = np.array(Made_Y)
    np.save('./Made_X/Made_X %s_%s' % (input_data_length, model_num), X)
    np.save('./Made_X/Made_Y %s_%s' % (input_data_length, model_num), Y)
    # np.save('./Made_X_high/Made_Y %s_%s' % (input_data_length, model_num), Y_high)



2020-01-16 XLM ohlcv.xlsx 965
2020-01-23 TRX ohlcv.xlsx 1620
2020-01-31 AMO ohlcv.xlsx 2232
2020-01-16 SNT ohlcv.xlsx 2409
len(dataX) < 100
2020-01-19 LBA ohlcv.xlsx 2409
2020-01-11 FX ohlcv.xlsx 2687
2020-01-22 LUNA ohlcv.xlsx 3748
2020-01-16 LINK ohlcv.xlsx 4704
len(dataX) < 100
2020-01-28 PLY ohlcv.xlsx 4704
2020-01-13 FCT ohlcv.xlsx 4871
2020-01-20 IPX ohlcv.xlsx 6123
len(dataX) < 100
2020-01-27 ITC ohlcv.xlsx 6123
2020-01-20 SNT ohlcv.xlsx 6335
2020-01-29 TMTG ohlcv.xlsx 7209
len(dataX) < 100
2020-01-25 VET ohlcv.xlsx 7209
2020-01-21 EOS ohlcv.xlsx 8331
2020-01-24 FAB ohlcv.xlsx 8458
2020-01-22 TRV ohlcv.xlsx 8609
2020-01-22 MTL ohlcv.xlsx 8946
2020-01-28 TRX ohlcv.xlsx 9549
len(dataX) < 100
2020-01-31 LAMB ohlcv.xlsx 9549
len(dataX) < 100
2020-01-20 CMT ohlcv.xlsx 9549
2020-01-22 BTG ohlcv.xlsx 10518
2020-01-13 DAC ohlcv.xlsx 11637
2020-01-25 REP ohlcv.xlsx 11911
2020-01-11 APIS ohlcv.xlsx 12101
2020-01-26 XRP ohlcv.xlsx 13129
2020-01-28 LINK ohlcv.xlsx 13485
2020-01-13 APIS ohlc

len(dataX) < 100
2020-01-25 VALOR ohlcv.xlsx 80823
2020-01-14 BCH ohlcv.xlsx 81562
2020-01-19 QTUM ohlcv.xlsx 82725
len(dataX) < 100
2020-01-12 BAT ohlcv.xlsx 82725
2020-01-19 BSV ohlcv.xlsx 84017
2020-01-30 NPXS ohlcv.xlsx 84152
2020-01-12 ENJ ohlcv.xlsx 84291
2020-01-24 BTG ohlcv.xlsx 85040
len(dataX) < 100
2020-01-20 ZRX ohlcv.xlsx 85040
2020-01-25 PIVX ohlcv.xlsx 85159
2020-01-23 BSV ohlcv.xlsx 86371
len(dataX) < 100
2020-01-25 NPXS ohlcv.xlsx 86371
2020-01-24 CON ohlcv.xlsx 87283
len(dataX) < 100
2020-01-30 STRAT ohlcv.xlsx 87283
2020-01-31 ADA ohlcv.xlsx 88134
2020-01-13 LUNA ohlcv.xlsx 88832
2020-01-12 ADA ohlcv.xlsx 89135
2020-01-12 AOA ohlcv.xlsx 89288
2020-01-21 WTC ohlcv.xlsx 89644
len(dataX) < 100
2020-01-11 HDAC ohlcv.xlsx 89644
2020-01-28 XRP ohlcv.xlsx 90790
2020-01-23 FCT ohlcv.xlsx 91001
2020-01-19 BCH ohlcv.xlsx 92300
2020-01-27 WPX ohlcv.xlsx 92554
2020-01-25 ETC ohlcv.xlsx 93421
2020-01-22 STEEM ohlcv.xlsx 94177
len(dataX) < 100
2020-01-16 CRO ohlcv.xlsx 94177
len(d

2020-01-23 TMTG ohlcv.xlsx 166513
len(dataX) < 100
2020-01-15 PAY ohlcv.xlsx 166513
2020-01-20 STEEM ohlcv.xlsx 167573
len(dataX) < 100
2020-01-21 BHP ohlcv.xlsx 167573
2020-01-22 WICC ohlcv.xlsx 167682
len(dataX) < 100
2020-01-23 LAMB ohlcv.xlsx 167682
len(dataX) < 100
2020-01-15 ELF ohlcv.xlsx 167682
2020-01-20 XLM ohlcv.xlsx 168361
2020-01-21 XSR ohlcv.xlsx 169050
2020-01-27 BCD ohlcv.xlsx 169303
len(dataX) < 100
2020-01-11 BAT ohlcv.xlsx 169303
len(dataX) < 100
2020-01-28 HC ohlcv.xlsx 169303
2020-01-15 DAC ohlcv.xlsx 170385
len(dataX) < 100
2020-01-13 BZNT ohlcv.xlsx 170385
len(dataX) < 100
2020-01-13 PPT ohlcv.xlsx 170385
len(dataX) < 100
2020-01-11 ZIL ohlcv.xlsx 170385
2020-01-30 STEEM ohlcv.xlsx 170630
len(dataX) < 100
2020-01-23 WOM ohlcv.xlsx 170630
2020-01-11 XLM ohlcv.xlsx 170964
2020-01-23 SOC ohlcv.xlsx 172053
len(dataX) < 100
2020-01-15 HC ohlcv.xlsx 172053
2020-01-13 HYC ohlcv.xlsx 172246
len(dataX) < 100
2020-01-24 KNC ohlcv.xlsx 172246
len(dataX) < 100
2020-01-19 SNT

2020-01-20 SOC ohlcv.xlsx 232044
len(dataX) < 100
2020-01-19 BAT ohlcv.xlsx 232044
2020-01-16 TMTG ohlcv.xlsx 232911
2020-01-14 EOS ohlcv.xlsx 234003
2020-01-16 PPT ohlcv.xlsx 234288
2020-01-24 APIS ohlcv.xlsx 234394
2020-01-26 AMO ohlcv.xlsx 234896
len(dataX) < 100
2020-01-19 THETA ohlcv.xlsx 234896
2020-01-29 ADA ohlcv.xlsx 235721
len(dataX) < 100
2020-01-12 PIVX ohlcv.xlsx 235721
len(dataX) < 100
2020-01-26 XEM ohlcv.xlsx 235721
len(dataX) < 100
2020-01-29 XEM ohlcv.xlsx 235721
len(dataX) < 100
2020-01-14 CTXC ohlcv.xlsx 235721
2020-01-30 ETH ohlcv.xlsx 236864
2020-01-19 REP ohlcv.xlsx 237576
len(dataX) < 100
2020-01-23 CTXC ohlcv.xlsx 237576
len(dataX) < 100
2020-01-20 BAT ohlcv.xlsx 237576
2020-01-19 BTG ohlcv.xlsx 238625
2020-01-20 ZEC ohlcv.xlsx 239054
len(dataX) < 100
2020-01-19 BHP ohlcv.xlsx 239054
len(dataX) < 100
2020-01-28 OMG ohlcv.xlsx 239054
len(dataX) < 100
2020-01-30 IOST ohlcv.xlsx 239054
len(dataX) < 100
2020-01-22 IOST ohlcv.xlsx 239054
len(dataX) < 100
2020-01-23 

2020-01-16 CON ohlcv.xlsx 303750
2020-01-24 FNB ohlcv.xlsx 304202
2020-01-20 TRX ohlcv.xlsx 305086
2020-01-15 XLM ohlcv.xlsx 305714
len(dataX) < 100
2020-01-19 CHR ohlcv.xlsx 305714
len(dataX) < 100
2020-01-25 FX ohlcv.xlsx 305714
len(dataX) < 100
2020-01-25 HDAC ohlcv.xlsx 305714
2020-01-20 VET ohlcv.xlsx 305888
2020-01-26 BSV ohlcv.xlsx 307005
2020-01-26 FNB ohlcv.xlsx 307550
len(dataX) < 100
2020-01-20 CRO ohlcv.xlsx 307550
len(dataX) < 100
2020-01-29 OMG ohlcv.xlsx 307550
2020-01-30 ICX ohlcv.xlsx 307734
2020-01-13 ENJ ohlcv.xlsx 307861
2020-01-22 XSR ohlcv.xlsx 308575
2020-01-16 STRAT ohlcv.xlsx 308830
len(dataX) < 100
2020-01-24 OGO ohlcv.xlsx 308830
2020-01-13 MTL ohlcv.xlsx 309147
2020-01-28 STEEM ohlcv.xlsx 309262
2020-01-23 AMO ohlcv.xlsx 309875
2020-01-31 DAD ohlcv.xlsx 310147
len(dataX) < 100
2020-01-24 ITC ohlcv.xlsx 310147
len(dataX) < 100
2020-01-15 ZRX ohlcv.xlsx 310147
len(dataX) < 100
2020-01-30 TRV ohlcv.xlsx 310147
len(dataX) < 100
2020-01-14 KNC ohlcv.xlsx 310147
2

2020-01-16 PAY ohlcv.xlsx 380549
2020-01-19 DAC ohlcv.xlsx 381769
len(dataX) < 100
2020-01-25 MCO ohlcv.xlsx 381769
len(dataX) < 100
2020-01-20 TRUE ohlcv.xlsx 381769
len(dataX) < 100
2020-01-13 WTC ohlcv.xlsx 381769
2020-01-22 TMTG ohlcv.xlsx 382771
len(dataX) < 100
2020-01-19 HYC ohlcv.xlsx 382771
2020-01-11 TRV ohlcv.xlsx 383063
len(dataX) < 100
2020-01-24 OMG ohlcv.xlsx 383063
2020-01-22 REP ohlcv.xlsx 383504
2020-01-11 BCD ohlcv.xlsx 384002
len(dataX) < 100
2020-01-22 MCO ohlcv.xlsx 384002
2020-01-15 TRV ohlcv.xlsx 384294
2020-01-13 IPX ohlcv.xlsx 384866
2020-01-28 TMTG ohlcv.xlsx 385753
2020-01-20 BTC ohlcv.xlsx 387059
2020-01-15 CON ohlcv.xlsx 387955
len(dataX) < 100
2020-01-22 MIX ohlcv.xlsx 387955
2020-01-20 QTUM ohlcv.xlsx 388786
2020-01-11 BTG ohlcv.xlsx 389163
len(dataX) < 100
2020-01-23 ORBS ohlcv.xlsx 389163
2020-01-16 ZRX ohlcv.xlsx 389329
2020-01-12 BTG ohlcv.xlsx 390195
2020-01-24 ETZ ohlcv.xlsx 390299
len(dataX) < 100
2020-01-14 VET ohlcv.xlsx 390299
2020-01-23 DAD oh

len(dataX) < 100
2020-01-31 XVG ohlcv.xlsx 446997
2020-01-16 VALOR ohlcv.xlsx 447166
2020-01-20 DASH ohlcv.xlsx 447511
len(dataX) < 100
2020-01-20 XEM ohlcv.xlsx 447511
2020-01-23 XRP ohlcv.xlsx 448685
2020-01-21 LINK ohlcv.xlsx 449174
len(dataX) < 100
2020-01-29 RNT ohlcv.xlsx 449174
2020-01-14 BTT ohlcv.xlsx 449362
2020-01-14 LUNA ohlcv.xlsx 450229
2020-01-15 TMTG ohlcv.xlsx 451176
len(dataX) < 100
2020-01-20 PIVX ohlcv.xlsx 451176
2020-01-29 XSR ohlcv.xlsx 452006
len(dataX) < 100
2020-01-31 SNT ohlcv.xlsx 452006
len(dataX) < 100
2020-01-21 OCN ohlcv.xlsx 452006
2020-01-29 BTC ohlcv.xlsx 453319
2020-01-20 NPXS ohlcv.xlsx 453466
2020-01-29 MBL ohlcv.xlsx 454627
2020-01-24 DASH ohlcv.xlsx 454734
2020-01-25 XEM ohlcv.xlsx 454846
len(dataX) < 100
2020-01-31 GXC ohlcv.xlsx 454846
2020-01-16 ZEC ohlcv.xlsx 455478
2020-01-20 LTC ohlcv.xlsx 456299
len(dataX) < 100
2020-01-24 MIX ohlcv.xlsx 456299
2020-01-31 PAY ohlcv.xlsx 456428
len(dataX) < 100
2020-01-14 IOST ohlcv.xlsx 456428
2020-01-29 X

2020-01-11 HYC ohlcv.xlsx 512835
2020-01-28 BTT ohlcv.xlsx 512988
len(dataX) < 100
2020-01-30 HC ohlcv.xlsx 512988
len(dataX) < 100
2020-01-27 ANKR ohlcv.xlsx 512988
len(dataX) < 100
2020-01-11 WTC ohlcv.xlsx 512988
2020-01-14 AMO ohlcv.xlsx 513639
2020-01-31 DASH ohlcv.xlsx 513984
len(dataX) < 100
2020-01-21 DVP ohlcv.xlsx 513984
len(dataX) < 100
2020-01-26 DASH ohlcv.xlsx 513984
len(dataX) < 100
2020-01-13 XMR ohlcv.xlsx 513984
len(dataX) < 100
2020-01-13 IOST ohlcv.xlsx 513984
2020-01-26 ETC ohlcv.xlsx 514576
len(dataX) < 100
2020-01-30 KNC ohlcv.xlsx 514576
len(dataX) < 100
2020-01-22 LAMB ohlcv.xlsx 514576
len(dataX) < 100
2020-01-15 HDAC ohlcv.xlsx 514576
len(dataX) < 100
2020-01-13 CMT ohlcv.xlsx 514576
len(dataX) < 100
2020-01-19 ITC ohlcv.xlsx 514576
2020-01-23 LTC ohlcv.xlsx 514996
2020-01-29 BCD ohlcv.xlsx 515811
len(dataX) < 100
2020-01-15 QKC ohlcv.xlsx 515811
2020-01-31 BCD ohlcv.xlsx 516304
2020-01-29 ETH ohlcv.xlsx 517487
len(dataX) < 100
2020-01-21 THETA ohlcv.xlsx 517